# Eurovision Song Contest

This notebook requires that the following files (from https://github.com/mantzaris/eurovision) are in the same directory:
- 1957.csv
- 1958.csv
- ...
- 2017.csv

## Import the libraries

In [ ]:
import numpy as np
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm, to_hex
from matplotlib.ticker import MultipleLocator
%matplotlib inline
# pandas
import pandas as pd

## Read the data

In [ ]:
df = pd.read_csv("1957.csv")
df.head()

In [ ]:
df = pd.read_csv("2017.csv")
df.head()

By chance we spot that Romania is misspelled, so we check for spelling errors

In [ ]:
for year in range(1957, 2018):
    df = pd.read_csv("{}.csv".format(year))
    countries1 = df['From country'].tolist()
    countries2 = df.columns.tolist()
    countries2.remove('From country')
    for c2 in countries2:
        if c2 not in countries1:
            print(year, c2)

Store all the data into a single data frame (and take care of the 'Romainia' spelling error) 

In [ ]:
esc_points = pd.DataFrame()
for year in range(1957, 2018):
    df = pd.read_csv("{}.csv".format(year))
    df['Year'] = year
    df.rename(index=str, columns={"Romainia": "Romania"})
    esc_points = esc_points.append(df, ignore_index=True, sort=True)
esc_points.head()

Move the columns 'Year' and 'From country' to the beginning

In [ ]:
cols = esc_points.columns.tolist()
cols.insert(0, cols.pop(cols.index('Year')))
cols.insert(1, cols.pop(cols.index('From country')))
esc_points = esc_points.reindex(columns=cols)
esc_points.head()

Save the table to a file

In [ ]:
esc_points.to_csv("ESC_points_1957-2017.csv", sep=';', na_rep='NaN')

## Competing countries through the years

Visualize how the number of competing countries changed through the years

In [ ]:
countries = esc_points[['From country', 'Year']].groupby('Year').count()
countries.head()

In [ ]:
countries.reset_index(inplace=True)
countries.columns = ['Year', 'Countries']
countries.head()

The first year (1956) of ESC is missing, add it manually

In [ ]:
countries.loc[-1] = [1956, 7]          # add a row
countries.index = countries.index + 1  # shift the index
countries = countries.sort_index()     # sort by index
countries.head()
# Note that if you run this cell more than once, you will get multiple rows with year 1956!

Save the table to a file

In [ ]:
esc_points.to_csv("ESC_countries_1956-2017.csv", sep=';')

### Visualize the number of countries

In [ ]:
ax = countries.plot.line(x='Year', y='Countries')
plt.show()

Refine the visualization:
- Create a bigger plot (with larger fonts)
- Choose a brighter color
- Remove the legend
- Adjust the y axis (to start at 0)
- Add a title

In [ ]:
categorical_map = plt.get_cmap('Set1') # Choose the red color from the 'Set1' categorical map
color = to_hex(categorical_map.colors[0])

In [ ]:
plt.rcParams.update({'font.size': 12})
ax = countries.plot.line(x='Year', y='Countries', figsize=(10,8), legend=None, c=color)
ax.set_ylim([0, 45])
ax.set_title('Number of countries competing in the Eurovision Song Contest')
plt.show()

Choose interesting years to be highlighted

In [ ]:
key_years = countries.loc[countries['Year'].isin([1956, 1970, 1993, 2004])]
key_years

In [ ]:
text = ['Only seven countries participated in 1956 (each with two songs,which never happened again)', 
        'Four countries did not participate in 1970 as a protest against rules allowing multiple winners in 1969', 
        'Qualifications were introduced in 1993 to limit the number of competing countries to 25', 
        'The introduction of a semi-final in 2004 enabled more countries to take part in the contest'
       ]
paragraph = ['Only seven countries participated\nin 1956 (each with two songs,\nwhich never happened again)',
             'Four countries did not participate\nin 1970 as a protest against rules\nallowing multiple winners in 1969',
             'Qualifications were introduced\nin 1993 to limit the number of\ncompeting countries to 25',
             'The introduction of a semi-final\nin 2004 enabled more countries\nto take part in the contest'
            ]
key_years = key_years.assign(Text=text, Paragraph=paragraph)
key_years

Add annotations and data source (and create a function so that is easy to invoke)

In [ ]:
title = 'Number of countries competing in the Eurovision Song Contest'
data_source = 'Data source: https://github.com/mantzaris/eurovision'

In [ ]:
def plot_countries():
    ax = countries.plot.line(x='Year', y='Countries', figsize=(10,8), legend=None, c=color)
    key_years.plot.scatter(ax=ax, x='Year', y='Countries', s=50, legend=None, c=color)
    ax.set_ylim([0, 45])
    ax.set_title(title)
    ann_x = key_years['Year'].tolist()
    ann_y = key_years['Countries'].tolist()
    ann_t = key_years['Paragraph'].tolist()
    ax.annotate(ann_t[0], 
                xy=(ann_x[0] + 0.5, ann_y[0]), 
                xytext=(30, 7), textcoords='offset points', 
                ha='left', va='top', 
                arrowprops=dict(color='k', arrowstyle='-|>', connectionstyle='arc,angleA=90,angleB=0,armB=40'))
    ax.annotate(ann_t[1], 
                xy=(ann_x[1] + 0.5, ann_y[1]), 
                xytext=(30, 22), textcoords='offset points', 
                ha='left', va='top', 
                arrowprops=dict(color='k', arrowstyle='-|>', connectionstyle='arc,angleA=90,angleB=0,armB=40'))
    ax.annotate(ann_t[2], 
                xy=(ann_x[2] - 0.5, ann_y[2]), 
                xytext=(-30, 22), textcoords='offset points', 
                ha='right', va='top', 
                arrowprops=dict(color='k', arrowstyle='-|>', connectionstyle='arc,angleA=0,angleB=180,armB=50'))
    ax.annotate(ann_t[3], 
                xy=(ann_x[3] - 0.5, ann_y[3]), 
                xytext=(-30, 22), textcoords='offset points', 
                ha='right', va='top', 
                arrowprops=dict(color='k', arrowstyle='-|>', connectionstyle='arc,angleA=0,angleB=180,armB=50'))
    ax.annotate(data_source,
                color='dimgray',
                xy=(0, 0), 
                xycoords='figure fraction',
                xytext=(30, 0),  
                textcoords='offset points',
                ha='left', va='bottom')
    plt.show()

In [ ]:
plot_countries()

Change style and try again

In [ ]:
import warnings
import matplotlib.cbook

def change_style(style_name='default'):
    warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation) # To suppres deprecation warnings
    plt.rcParams.update(plt.rcParamsDefault) # To reset the style
    # To restart the inline backend 
    %matplotlib inline                       
    plt.style.use(style_name)

In [ ]:
change_style('ggplot')
plot_countries()

In [ ]:
change_style('seaborn')
plot_countries()

In [ ]:
change_style('fivethirtyeight')
plot_countries()

### Add interactivity

#### Plotly

In [ ]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

A useful Plotly cheatsheet: https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf

In [ ]:
data = [
    go.Scatter(
        x=countries['Year'], 
        y=countries['Countries']
    )
]
iplot(data)

Refine the visualization:
- Choose a brighter color
- Label the axes and adjust their ranges
- Add a title

In [ ]:
data = [
    go.Scatter(
        x=countries['Year'], 
        y=countries['Countries'], 
        line=dict(color=color)
    )
]
layout = dict(
    title=title,
    xaxis=dict(title='Year', range=[1955, 2020]),
    yaxis=dict(title='Countries', range=[0, 45])
)
iplot(dict(data=data, layout=layout))

Add annotations

In [ ]:
data = [
    go.Scatter(
        x=countries['Year'], 
        y=countries['Countries'], 
        line=dict(color=color),
        hoverinfo='x+y'
    ),
    go.Scatter(
        x=key_years['Year'],
        y=key_years['Countries'],
        mode='markers',
        marker=dict(color=color, size=8),
        hoverinfo='text',
        hovertext=key_years['Text'].tolist())
]
layout = dict(
    title=title,
    xaxis=dict(title='Year', range=[1955, 2020]),
    yaxis=dict(title='Countries', range=[0, 45]),
    showlegend=False,
    annotations=[
        dict(xref='paper', x=-0.08,
             yref='paper', y=-0.22,
             text=data_source,
             showarrow=False)
    ]
)
iplot(dict(data=data, layout=layout))

#### Bokeh

In [ ]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.models import HoverTool 

In [ ]:
output_notebook()

In [ ]:
p = figure()
p.line(x='Year', y='Countries', source=countries)
show(p)

Refine the visualization:
- Choose a brighter color
- Label the axes and adjust their ranges
- Add a title

In [ ]:
p = figure(title=title, 
           x_axis_label='Year', 
           y_axis_label='Countries',
           y_range=(0, 44))
p.line(x='Year', y='Countries', source=countries, color=color)
show(p)

Add annotations

In [ ]:
p = figure(title=title, 
           x_axis_label='Year', 
           y_axis_label='Countries',
           y_range=(0, 44),
          )
tooltips_line = [
    ("Year", "$x{(0.)}"),
    ("Countries", "$y{(0.)}")
]
tooltips_circle = [
    ("Text", "$Text"),
]
bl = p.line(x='Year', y='Countries', source=countries, color=color)
bc = p.circle(x='Year', y='Countries', source=ColumnDataSource(key_years), size=6, color=color)
hover_line = HoverTool(renderers=[bl], tooltips=tooltips_line)
hover_circle = HoverTool(renderers=[bc], tooltips=tooltips_circle)
p.add_tools(hover_line, hover_circle)
show(p)